<a href="https://colab.research.google.com/github/FedirAnichkin/FedirAnichkin.github.io/blob/master/colab_files/Correlation_between_trial_finished_paid.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Завдання**.

Дослідити, чи існує кореляція між дозвоном робота Voximplant, прочитанням листів, переглядом повідомлення та приходом на пробний (та в оплату), порівняти їх з користувачами, яким не було дзвінка/не прочитали лист.
Лінк на тікет - [тут](https://https://allright.myjetbrains.com/youtrack/issue/BI-1343/Korelyaciya-mizh-prihodom-na-probnij-ta-oplatu-ta-prochitanim-povidomlennyam)

# Аутентифікація користувача в Google, імпорт потрібних бібліотек та приєднання до BigQuery

In [1]:
# Authenticate user to google cloud with your current credentials
from google.colab import auth
auth.authenticate_user()

In [2]:
# imports BigQuery, DataFrames and syntax for sql
from google.cloud import bigquery
import bigframes.pandas as bpd
from google.colab import syntax
import pandas as pd
from functools import reduce
import plotly.express as px

# connecting to BigQuery
project_id = 'allright-234310'
client = bigquery.Client(project=project_id)

In [3]:
# Set BigQuery DataFrames options
bpd.options.bigquery.project = project_id
bpd.options.bigquery.location = "US"

# Формування запиту до BigQuery та перегляд витягнутої інформації

In [4]:
query = syntax.sql('''
  SELECT *
  FROM `allright-234310.data_science.corelation_between_voxcall_opened_email_and_presenting_on_trial_table`
  WHERE date ='2025-06-01'
''')

# Load data from a BigQuery table using BigFrames DataFrames:
data_raw = bpd.read_gbq(query)
data = data_raw.to_pandas()

In [5]:
data['date'] = pd.to_datetime(data['date'], errors='coerce')
# data = data[data['locale']== 'pl']
data.sort_values(by=['date', 'free_student_id'], ascending=[False, False]).reset_index(drop=True).head(5)

,date,free_student_id,locale,reg_country,present_student,vox_call_id,vox_not_call_id,email_opened,paid_id,first_amount_usd,...,saw_video,faq_read,child_study_english_before,child_had_device,plan_two_lesson,plan_three_lesson,plan_four_lesson,unitalk_call,unitalk_not_call,message_read
0,2025-06-01,1102439,it,Italy,<NA>,<NA>,<NA>,1102439,<NA>,<NA>,...,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>
1,2025-06-01,1102438,uk,Switzerland,1102438,<NA>,<NA>,<NA>,<NA>,<NA>,...,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>
2,2025-06-01,1102437,uk,Ukraine,1102437,<NA>,<NA>,<NA>,<NA>,<NA>,...,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>
3,2025-06-01,1102436,it,Italy,1102436,<NA>,<NA>,1102436,<NA>,<NA>,...,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>
4,2025-06-01,1102435,uk,Ukraine,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,...,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>


# Блок 1. Кореляція між дозвонами, читанням листів, виконання placement test, тестування зв'язку та приходом на пробний, оплатою і побудова кореляціних матриць.

## Опрацювання даних

In [6]:
all_students_conversion = data.copy()
all_students_conversion.head()

,date,free_student_id,locale,reg_country,present_student,vox_call_id,vox_not_call_id,email_opened,paid_id,first_amount_usd,...,saw_video,faq_read,child_study_english_before,child_had_device,plan_two_lesson,plan_three_lesson,plan_four_lesson,unitalk_call,unitalk_not_call,message_read
0,2025-06-01,1099763,ru,Portugal,1099763,<NA>,<NA>,<NA>,1099763,278.096626,...,<NA>,<NA>,1099763,1099763,1099763,<NA>,<NA>,<NA>,<NA>,<NA>
1,2025-06-01,1095620,he,Israel,1095620,<NA>,<NA>,<NA>,<NA>,<NA>,...,<NA>,<NA>,<NA>,1095620,1095620,<NA>,<NA>,<NA>,<NA>,1095620
2,2025-06-01,1099939,pl,Poland,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,...,<NA>,<NA>,1099939,<NA>,1099939,1099939,<NA>,<NA>,<NA>,<NA>
3,2025-06-01,1102239,uk,Ukraine,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,...,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>
4,2025-06-01,1099821,pl,Poland,1099821,<NA>,<NA>,1099821,<NA>,<NA>,...,<NA>,<NA>,1099821,<NA>,1099821,<NA>,<NA>,<NA>,<NA>,<NA>


## Побудова матриці 1 з розрахованими даними

In [7]:
matrix_data = all_students_conversion[['locale', 'child_study_english_before', 'child_had_device', 'plan_two_lesson', 'plan_three_lesson', 'plan_four_lesson', 'email_opened', 'vox_call_id', 'vox_not_call_id', 'unitalk_call', 'unitalk_not_call', 'placement_test_user_id',
                                       'test_connection_user_id', 'added_date', 'message_read', 'present_student']]

In [8]:
columns = ['child_study_english_before', 'child_had_device', 'plan_two_lesson', 'plan_three_lesson', 'plan_four_lesson', 'email_opened', 'vox_call_id', 'vox_not_call_id', 'unitalk_call', 'unitalk_not_call', 'placement_test_user_id',
                                       'test_connection_user_id', 'added_date', 'message_read',  'present_student']
matrix_data.loc[:, columns] = matrix_data.loc[:, columns].notnull().astype(int)
matrix_data_trans = matrix_data.rename(columns={"vox_call_id": "was robot call", "vox_not_call_id": "was not robot call", "unitalk_call": "call by admin", "unitalk_not_call": "try to call by admin", "placement_test_user_id": "was_placement_test", "test_connection_user_id": "test_connection", "present_student": "finished lesson"})

In [9]:
locales = matrix_data_trans['locale'].unique()
correlation_matrices = {}

for locale in locales:
    locale_data = matrix_data_trans[matrix_data_trans['locale'] == locale].drop(columns=['locale'])
    correlation_matrices[locale] = locale_data.corr().round(3)

## Візуалізація кореляційної матриці признаків для кожної локалі, що можуть впливати на прихід на пробний

In [10]:
for locale, matrix in correlation_matrices.items():
    fig = px.imshow(matrix, text_auto=True, title=f'Кореляційна матриця признаків, що можуть впливати на прихід на пробний для локалі: {locale}',  color_continuous_scale=px.colors.sequential.Greys, aspect="auto")
    fig.update_xaxes(side="top")
    fig.show()

На матрицях вище можемо побачити, що дзвінок адміна, прочитання листа(ів), проходження плейсмент тесту та перевірка з'єднання мають найбільший вплив на прихід на пробний.

## Побудова матриці 2 з розрахованими даними

In [11]:
matrix_data_paid = all_students_conversion[['locale', 'child_study_english_before', 'child_had_device', 'plan_two_lesson', 'plan_three_lesson', 'plan_four_lesson', 'email_opened', 'vox_call_id', 'vox_not_call_id', 'unitalk_call', 'unitalk_not_call', 'placement_test_user_id',
                                       'test_connection_user_id', 'added_date', 'message_read',  'paid_id']]

In [12]:
columns = ['child_study_english_before', 'child_had_device', 'plan_two_lesson', 'plan_three_lesson', 'plan_four_lesson', 'email_opened', 'vox_call_id', 'vox_not_call_id', 'unitalk_call', 'unitalk_not_call', 'placement_test_user_id',
                                       'test_connection_user_id', 'added_date', 'message_read',  'paid_id']
matrix_data_paid.loc[:, columns] = matrix_data_paid.loc[:, columns].notnull().astype(int)
matrix_data_paid_trans = matrix_data_paid.rename(columns={"vox_call_id": "was call", "vox_not_call_id": "was not vox call", "placement_test_user_id": "was_placement_test", "unitalk_call": "call by admin", "unitalk_not_call": "try to call by admin", "test_connection_user_id": "test_connection", "paid_id": "paid"})

In [13]:
locales_paid = matrix_data_paid_trans['locale'].unique()
correlation_matrices_paid = {}

for locale in locales_paid:
    locale_data = matrix_data_paid_trans[matrix_data_paid_trans['locale'] == locale].drop(columns=['locale'])
    correlation_matrices_paid[locale] = locale_data.corr().round(3)

## Візуалізація кореляційної матриці признаків для кожної локалі, що можуть впливати на оплату

In [14]:
for locale, matrix in correlation_matrices_paid.items():
    fig = px.imshow(matrix, text_auto=True, title=f'Кореляційна матриця признаків, що можуть впливати на оплату для локалі: {locale}',  color_continuous_scale=px.colors.sequential.Greys, aspect="auto")
    fig.update_xaxes(side="top")
    fig.show()

Як і для приходу на пробний, на оплату також можуть впливати дзвінок адміна, прочитання листа(ів), проходження плейсмент тесту та перевірка з'єднання.

Перевіримо відсоток користувачів по кожному признаку, що може впливати на прихід на пробний та на оплату.

In [15]:
data_raw = all_students_conversion.copy()
columns = ['email_opened', 'placement_test_user_id', 'test_connection_user_id', 'unitalk_call', 'unitalk_not_call', 'message_read']
data_raw.loc[:, columns] = data_raw[columns].notnull().astype(int)
result = data_raw.groupby('locale').agg(
    count=('free_student_id', 'nunique'),
    email_opened=('email_opened', 'sum'),
    placement_test_maded=('placement_test_user_id', 'sum'),
    test_connection_maded=('test_connection_user_id', 'sum'),
    call_by_admin=('unitalk_call', 'sum'),
    try_to_call_by_admin=('unitalk_not_call', 'sum'),
    message_read=('message_read', 'sum')
)
result['% email_read'] = ((result['email_opened'] / result['count']) * 100).round(2)
result['% placement_test_maded'] = ((result['placement_test_maded'] / result['count']) * 100).round(2)
result['% test_connection_maded'] = ((result['test_connection_maded'] / result['count']) * 100).round(2)
result['% call_by_admin'] = ((result['call_by_admin'] / result['count']) * 100).round(2)
result['% try_to_call_by_admin'] = ((result['try_to_call_by_admin'] / result['count']) * 100).round(2)
result['% message_read'] = ((result['message_read'] / result['count']) * 100).round(2)
result = result[['count', '% email_read', '% placement_test_maded', '% test_connection_maded', '% call_by_admin', '% try_to_call_by_admin', '% message_read']].sort_values(by=['count'], ascending=[False])

In [16]:
result

,count,% email_read,% placement_test_maded,% test_connection_maded,% call_by_admin,% try_to_call_by_admin,% message_read
locale,,,,,,,
uk,1587,33.77,0.0,10.96,0.0,0.0,0.38
pl,361,36.01,0.0,19.67,9.7,1.94,14.96
he,340,21.47,0.0,17.06,0.0,0.0,15.29
it,333,36.94,0.0,10.51,0.0,0.0,14.71
ro,318,32.39,0.0,7.86,0.0,0.0,22.96
latam,245,26.53,0.0,2.45,0.0,0.0,4.49
ru,193,29.02,0.0,13.47,0.0,0.0,0.0
ar,170,20.0,0.0,12.35,14.12,10.59,11.76
sr,126,43.65,0.0,10.32,0.0,0.0,20.63


# Блок 2. Конверсії в прихід на пробний в залежності від прочитання листа(ів), перевірка з'єднання, перегляд повідомлення та дзвінка від адміна.

In [23]:
all_students_conversion = data.copy().sort_values(by=['date', 'free_student_id'], ascending=[False, False]).reset_index(drop=True)
data_raw_trial = all_students_conversion.copy()
result_all = data_raw_trial.groupby('locale').agg(
    booked_lessons=('free_student_id', 'nunique'),
    finished_lessons=('present_student', 'nunique')
)
result_all['Trial Booked => Trial Finished'] = ((result_all['finished_lessons'] / result_all['booked_lessons']) * 100).round(2)
result_all = result_all[['booked_lessons', 'finished_lessons', 'Trial Booked => Trial Finished']].sort_values(by=['finished_lessons'], ascending=[False])

In [24]:
students_w_email = all_students_conversion[all_students_conversion['free_student_id'].isin(all_students_conversion['email_opened'])]
students_w_email_agg = students_w_email.groupby('locale', as_index=False )[['free_student_id', 'present_student']].nunique().sort_values(by='locale', ascending=False)
students_w_email_agg['% CR email read'] = (students_w_email_agg['present_student']/students_w_email_agg['free_student_id']).round(4)*100

students_w_message_read = all_students_conversion[all_students_conversion['free_student_id'].isin(all_students_conversion['message_read'])]
students_w_message_read_agg = students_w_message_read.groupby('locale', as_index=False )[['free_student_id', 'present_student']].nunique().sort_values(by='locale', ascending=False)
students_w_message_read_agg['% CR message_read'] = (students_w_message_read_agg['present_student']/students_w_message_read_agg['free_student_id']).round(4)*100

students_w_connection = all_students_conversion[all_students_conversion['free_student_id'].isin(all_students_conversion['test_connection_user_id'])]
students_w_connection_agg = students_w_connection.groupby('locale', as_index=False )[['free_student_id', 'present_student']].nunique().sort_values(by='locale', ascending=False)
students_w_connection_agg['% CR test connection'] = (students_w_connection_agg['present_student']/students_w_connection_agg['free_student_id']).round(4)*100

students_w_admin_call = all_students_conversion[all_students_conversion['free_student_id'].isin(all_students_conversion['unitalk_call'])]
students_w_admin_call_agg = students_w_admin_call.groupby('locale', as_index=False )[['free_student_id', 'present_student']].nunique().sort_values(by='locale', ascending=False)
students_w_admin_call_agg['% CR call by admin'] = (students_w_admin_call_agg['present_student']/students_w_admin_call_agg['free_student_id']).round(4)*100
students_w_admin_call_agg = students_w_admin_call_agg.drop(columns=['free_student_id', 'present_student'])

In [30]:
dfs = [result_all, students_w_email_agg, students_w_message_read_agg, students_w_connection_agg, students_w_admin_call_agg]
result = reduce(lambda left, right: pd.merge(left, right, on='locale', how='left'), dfs)
result[['locale', 'booked_lessons', 'finished_lessons', 'Trial Booked => Trial Finished', '% CR email read', '% CR message_read', '% CR test connection', '% CR call by admin']]

,locale,booked_lessons,finished_lessons,Trial Booked => Trial Finished,% CR email read,% CR message_read,% CR test connection,% CR call by admin
0,uk,1587,754,47.51,59.70,100.0,91.38,NaN
1,he,340,205,60.29,64.38,100.0,94.83,NaN
2,pl,361,202,55.96,70.00,100.0,88.73,100.0
3,it,333,175,52.55,65.85,100.0,97.14,NaN
4,ro,318,162,50.94,64.08,100.0,92.00,NaN
5,ru,193,107,55.44,76.79,NaN,100.00,NaN
6,latam,245,73,29.80,36.92,100.0,83.33,NaN
7,ar,170,62,36.47,36.36,100.0,52.38,100.0
8,sr,126,50,39.68,47.27,100.0,92.31,NaN
9,sk,78,42,53.85,65.91,100.0,100.00,100.0


# Блок 3. Конверсії в оплату в залежності від прочитання листа(ів), перевірка з'єднання, перегляд повідомлення та дзвінка від адміна.

In [31]:
data_raw_paid = all_students_conversion.copy()
result_all_paid = data_raw_paid.groupby('locale').agg(
    booked_lessons=('free_student_id', 'nunique'),
    finished_lessons=('present_student', 'nunique'),
    paids=('paid_id', 'nunique')
)
result_all_paid['Trial Booked => Paid'] = ((result_all_paid['paids'] / result_all_paid['booked_lessons']) * 100).round(2)
result_all_paid['Trial Finished => Paid'] = ((result_all_paid['paids'] / result_all_paid['finished_lessons']) * 100).round(2)
result_all_paid = result_all_paid[['booked_lessons', 'finished_lessons', 'paids', 'Trial Booked => Paid', 'Trial Finished => Paid']].sort_values(by=['paids'], ascending=[False])

In [32]:
students_w_email_paid = all_students_conversion[all_students_conversion['free_student_id'].isin(all_students_conversion['email_opened'])]
students_w_email_paid_agg = students_w_email_paid.groupby('locale', as_index=False )[['free_student_id', 'paid_id']].nunique().sort_values(by='locale', ascending=False)
students_w_email_paid_agg['% CR email read'] = (students_w_email_paid_agg['paid_id']/students_w_email_paid_agg['free_student_id']).round(4)*100

students_w_message_read_paid = all_students_conversion[all_students_conversion['free_student_id'].isin(all_students_conversion['message_read'])]
students_w_message_read_paid_agg = students_w_message_read_paid.groupby('locale', as_index=False )[['free_student_id', 'paid_id']].nunique().sort_values(by='locale', ascending=False)
students_w_message_read_paid_agg['% CR message_read'] = (students_w_message_read_paid_agg['paid_id']/students_w_message_read_paid_agg['free_student_id']).round(4)*100

students_w_connection_paid = all_students_conversion[all_students_conversion['free_student_id'].isin(all_students_conversion['test_connection_user_id'])]
students_w_connection_paid_agg = students_w_connection_paid.groupby('locale', as_index=False )[['free_student_id', 'paid_id']].nunique().sort_values(by='locale', ascending=False)
students_w_connection_paid_agg['% CR test connection'] = (students_w_connection_paid_agg['paid_id']/students_w_connection_paid_agg['free_student_id']).round(4)*100

students_w_admin_call_paid = all_students_conversion[all_students_conversion['free_student_id'].isin(all_students_conversion['unitalk_call'])]
students_w_admin_call_paid_agg = students_w_admin_call_paid.groupby('locale', as_index=False )[['free_student_id', 'paid_id']].nunique().sort_values(by='locale', ascending=False)
students_w_admin_call_paid_agg['% CR call by admin'] = (students_w_admin_call_paid_agg['paid_id']/students_w_admin_call_paid_agg['free_student_id']).round(4)*100
students_w_admin_call_paid_agg = students_w_admin_call_paid_agg.drop(columns=['free_student_id', 'paid_id'])

In [34]:
dfs_paid = [result_all_paid, students_w_email_paid_agg, students_w_message_read_paid_agg, students_w_connection_paid_agg, students_w_admin_call_paid_agg]
result_paid = reduce(lambda left, right: pd.merge(left, right, on='locale', how='left'), dfs_paid)
result_paid[['locale', 'booked_lessons', 'finished_lessons', 'paids', 'Trial Booked => Paid', 'Trial Finished => Paid', '% CR email read', '% CR message_read', '% CR test connection', '% CR call by admin']]

,locale,booked_lessons,finished_lessons,paids,Trial Booked => Paid,Trial Finished => Paid,% CR email read,% CR message_read,% CR test connection,% CR call by admin
0,uk,1587,754,159,10.02,21.09,12.50,16.67,27.01,NaN
1,pl,361,202,34,9.42,16.83,16.92,31.48,16.90,14.29
2,it,333,175,28,8.41,16.00,13.82,12.24,34.29,NaN
3,ro,318,162,25,7.86,15.43,8.74,13.70,20.00,NaN
4,ru,193,107,24,12.44,22.43,17.86,NaN,26.92,NaN
5,he,340,205,22,6.47,10.73,10.96,9.62,12.07,NaN
6,sk,78,42,8,10.26,19.05,13.64,22.73,16.67,22.22
7,ar,170,62,8,4.71,12.90,6.06,21.05,9.52,16.67
8,en,88,25,6,6.82,24.00,12.50,0.00,33.33,NaN
9,latam,245,73,5,2.04,6.85,3.08,0.00,0.00,NaN
